#### 사용자 정의 모델 클래스
- 부모클래스 : nn.Module
- 필수오버라이딩
    * _ _init_ _() : 모델 층 구성 즉, 설계
    * forward() : 순방향 학습 진행 코드 구현

In [59]:
# 모듈 로딩
import torch                                # 텐서 및 수치 계산 함수 관련 모듈
import torch.nn as nn                       # 인공신경망 관련 모듈
import torch.nn.functional as F             # 인공신경망 관련 모듈 (손실함수, 활성화함수 등등)
import torch.optim as optimizer             # 최적화 관련 모듈 (가중치, 절편 빠르게 찾아주는 알고리즘)
from torchmetrics.regression import R2Score # 회귀성능지표 관련 모듈
from torchmetrics.classification import *   # 분류성능지표 관련 모듈
from torchinfo import summary               # 모델 구조 및 정보 관련 모듈


import pandas as pd                         # 데이터 파일 분석 관련 모듈
from sklearn.model_selection import train_test_split

In [60]:
# 랜덤 고정
torch.manual_seed(10)

# 텐서 저장 및 실행 위치 설정
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'

- [기본] 신경망클래스 <hr>
    * 입력층 - 입력 : 피쳐수 고정
    * 출력증 - 출력 : 타겟수 고정
    * 은닉층 - 고정

In [61]:
# 모델설계
# 데이터셋 : 피쳐 4개, 타겟 1개, 회귀
# 입력층 : 입력   4개   출력   20개   AF ReLU
# 은닉층 : 입력  20개   출력  100개   AF ReLU
# 출력층 : 입력 100개   출력    1개   AF X, Sigmoid & softmax

In [62]:
class MyModel(nn.Module):
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 Callback func)
    def __init__(self):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer=nn.Linear(4,20) # W 4 + b 1 => 1p, 5 * 20 = 100개 변수
        self.hidden_layer=nn.Linear(20,100) # W 20 b + 1 => 21 * 100 = 2100개 변수
        self.output_layer=nn.Linear(100,1) # W 100 b + 1 => 101 * 1 =101개 변수

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y=self.input_layer(x) # 1개 퍼셉트론 : y=x1w1 + x2w2 + x3w3 + x4w4+b
        y=F.relu(y) # 0 <= y ----> 죽은 relu ==> leakyReLU

        y=self.hidden_layer(y) # 1개 퍼셉트론 : y=x1w1 + x2w2 +...+ x20w20 + b
        y=F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y=x1w1 + x2w2 +...+ x100w100 + b


In [63]:
# 입력 피쳐 수가 동적인 모델
class MyModel2(nn.Module):
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 Callback func)
    def __init__(self, in_feature):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer=nn.Linear(in_feature,20) # W 4 + b 1 => 1p, 5 * 20 = 100개 변수
        self.hidden_layer=nn.Linear(20,100) # W 20 b + 1 => 21 * 100 = 2100개 변수
        self.output_layer=nn.Linear(100,1) # W 100 b + 1 => 101 * 1 =101개 변수

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y=self.input_layer(x) # 1개 퍼셉트론 : y=x1w1 + x2w2 + x3w3 + x4w4+b
        y=F.relu(y) # 0 <= y ----> 죽은 relu ==> leakyReLU

        y=self.hidden_layer(y) # 1개 퍼셉트론 : y=x1w1 + x2w2 +...+ x20w20 + b
        y=F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y=x1w1 + x2w2 +...+ x100w100 + b


In [64]:
# 입력 피쳐 수, 은닉층 퍼셉트론 수 동적인 모델
class MyModel3(nn.Module):
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 Callback func)
    def __init__(self, in_feature, in_out, h_out):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer=nn.Linear(in_feature,in_out)
        self.hidden_layer=nn.Linear(in_out,h_out)
        self.output_layer=nn.Linear(h_out,1)

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y=self.input_layer(x) # 1개 퍼셉트론 : y=x1w1 + x2w2 + x3w3 + x4w4+b
        y=F.relu(y) # 0 <= y ----> 죽은 relu ==> leakyReLU

        y=self.hidden_layer(y) # 1개 퍼셉트론 : y=x1w1 + x2w2 +...+ x20w20 + b
        y=F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y=x1w1 + x2w2 +...+ x100w100 + b


In [68]:
# 인스턴스 생성
m1=MyModel3(4, 50, 30)
m1

MyModel3(
  (input_layer): Linear(in_features=4, out_features=50, bias=True)
  (hidden_layer): Linear(in_features=50, out_features=30, bias=True)
  (output_layer): Linear(in_features=30, out_features=1, bias=True)
)

In [66]:
# 모델 파라미터 즉, W와 b 확인
for m in m1.named_parameters(): print(m)

('input_layer.weight', Parameter containing:
tensor([[-0.0419, -0.0171, -0.1875,  0.1150],
        [-0.2861, -0.0882,  0.1938,  0.4693],
        [ 0.1178, -0.1696,  0.0479, -0.0560],
        [ 0.2041,  0.0573,  0.1959,  0.4849],
        [-0.2076, -0.0177,  0.1150, -0.0033],
        [-0.0479, -0.4425, -0.4313, -0.4499],
        [-0.4892, -0.4657, -0.3788, -0.4510],
        [-0.4690,  0.2192,  0.3067,  0.3379],
        [ 0.2694,  0.1694,  0.2203, -0.2765],
        [ 0.4502, -0.0345,  0.4314,  0.1533],
        [ 0.3914,  0.3988, -0.1045, -0.1454],
        [ 0.0752, -0.0213,  0.0782,  0.2536],
        [-0.3907, -0.0229, -0.3924,  0.4829],
        [-0.3517,  0.0956, -0.1366,  0.2842],
        [ 0.0017, -0.0503,  0.3660,  0.4567],
        [-0.3629, -0.4823,  0.0417,  0.1575],
        [ 0.1141,  0.4619,  0.2244, -0.2300],
        [-0.3424,  0.3879,  0.4792, -0.2373],
        [-0.3200,  0.1750, -0.3576, -0.1210],
        [-0.4945,  0.1368, -0.1705, -0.2797],
        [-0.3179, -0.1759,  0.2375,

In [67]:
# 학습진행 => 모델인스턴스명(데이터)
# 임의의 데이터
dataTS=torch.FloatTensor([[1,3,5,7],[2,4,6,8]])
targetTS=torch.FloatTensor([[4],[5]])

# 학습
pre_y=m1(dataTS)

print(pre_y)

calling forward()
tensor([[-0.2975],
        [-0.3586]], grad_fn=<AddmmBackward0>)
